In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
!pip install fuzzywuzzy

In [0]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
import numpy as np
from fuzzywuzzy import fuzz 
import ipywidgets as widgets
import pprint
from ipywidgets import interact, interact_manual
import re
__PATH__ = "/content/drive/My Drive/Lab/data.csv"

In [7]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [0]:
df = pd.read_csv(__PATH__,sep=";",header=0)

In [9]:
df.head()

,id,updatedDate,publishedDate,title,summary,authors,category,metaData,downloadLink,filePath
0,http://arxiv.org/abs/1407.6950v1,2014-07-24T16:56:39Z,2014-07-24T16:56:39Z,"How,whenAndHowMuchACardDeckIsWellShuffled.pdf",The Thesis Consider The Mixing Of Few 3 4 ...,Benjamin Isac Fargion,cs.DM,"Italian Thesis In Engeenering Computer, 26 Feb...",http://arxiv.org/pdf/1407.6950v1.pdf,"./files/How,whenAndHowMuchACardDeckIsWellShuff..."
1,http://arxiv.org/abs/0907.0618v1,2009-07-03T12:35:10Z,2009-07-03T12:35:10Z,QuantumIsometryGroups.pdf,This Thesis Contains The Formulation And Com...,Jyotishman Bhowmick,math.OA,Thesis,http://arxiv.org/pdf/0907.0618v1.pdf,./files/QuantumIsometryGroups.pdf
2,http://arxiv.org/abs/1806.09601v2,2018-07-14T17:06:27Z,2018-06-25T17:55:59Z,ComputationAndBoundingOfFolkmanNumbers.pdf,Phd Thesis Under The Supervision Of Professo...,Aleksandar Bikov,math.CO,PhD Thesis,http://arxiv.org/pdf/1806.09601v2.pdf,./files/ComputationAndBoundingOfFolkmanNumbers...
3,http://arxiv.org/abs/1905.03014v1,2019-05-08T11:47:34Z,2019-05-08T11:47:34Z,OnChurch'sThesisInCubicalAssemblies.pdf,"We Show That Church's Thesis, The Axiom Stat...","Andrew Swan, Taichi Uemura,",math.LO,0,http://arxiv.org/pdf/1905.03014v1.pdf,./files/OnChurch'sThesisInCubicalAssemblies.pdf
4,http://arxiv.org/abs/1901.04911v1,2019-01-15T16:24:07Z,2019-01-15T16:24:07Z,UnconstrainedChurchTuringThesisCannotPossiblyB...,The Church Turing Thesis Asserts That If A P...,Yuri Gurevich,cs.LO,0,http://arxiv.org/pdf/1901.04911v1.pdf,./files/UnconstrainedChurchTuringThesisCannotP...


In [0]:
titles = list(df['title'].apply(
    lambda t : 
        tuple(
            filter(lambda e:not e in stopwords.words('english'),
                map(lambda e:e.lower(),
                       re.findall('([A-Z]{1}[a-z]+)',t.replace('.pdf','')))
                )
            )
        )
    )

In [0]:
res = {}
for title in titles:
    synsets = {}
    for word in title:
        synsets[word]=[synset for synset in wn.synsets(word)]
    res[title] = synsets

In [0]:
def get_hypernyms_lvl_1(synset):
    hyps = set()
    while True:
        try:
            synset = synset.hypernyms()[-1]
            hyps.add(synset)
        except IndexError:
            break
    return hyps

In [0]:
def get_hypernyms_lvl_2(synset):
    hyps = dict()
    lvl_calc = 1
    while lvl_calc < 3:
        try:
            synset = synset.hypernyms()[0]
            hyps[lvl_calc] = synset 
            lvl_calc += 1
        except IndexError:
            break
    return hyps

In [0]:
def extract_hypornyms_en(a):
    hypornyms_en_1 = set()
    hypornyms_en_2 = set()
    for word in a:
        try:
            word_synset0 = wn.synsets(word)[0]
            hyps_buff = get_hypernyms_lvl_2(word_synset0)
            hypornyms_en_1.add(hyps_buff[1].name().split('.')[0])
            hypornyms_en_2.add(hyps_buff[2].name().split('.')[0])
        except IndexError:
            continue
        except KeyError:
            continue
    return hypornyms_en_1, hypornyms_en_2

In [0]:
def f1measure(a, b):
    a = set(a)
    b = set(b)
    overlap = set()
    overlap_hyp_cnt = 0
    overlap = a.intersection(b)
    overlap_hyp_cnt = len(overlap) 
    
    recl = overlap_hyp_cnt/len(a)
    prec = overlap_hyp_cnt/len(b)
    
    if len(overlap) == 0:
        return 0, overlap
    else:
        return 2*recl*prec/(recl+prec), overlap

In [16]:
lang = 'eng'

def divergence(a,b):
       
    a = set(a) 
    b = set(b) 
    f1score, overlap = f1measure(a, b)
    a = a - overlap
    b = b - overlap
    
    if len(a) == 0 or len(b) == 0:
        return 1.0 - f1score
    
    buff_a = {}
    buff_b = {}
    
    if lang == 'eng':
        buff_a_first, buff_a_second = extract_hypornyms_en(a)
        buff_b_first, buff_b_second = extract_hypornyms_en(b)
        
    overlap_hyp_cnt = 0
    print('buff_a_first',buff_a_first)
    print('buff_b_first',buff_b_first)
    print('buff_a_second',buff_a_second)
    print('buff_b_second',buff_b_second)
      
    if len(buff_a_first.intersection(buff_b_first)) > 0:
        overlap_hyp_cnt += 1
    
    if len(buff_a_second.intersection(buff_b_second)) > 0:
        overlap_hyp_cnt += 0.5
    
    
    
    recl_hyp = overlap_hyp_cnt/len(a)
    print('recl_hyp',recl_hyp)
    prec_hyp = overlap_hyp_cnt/len(b)
    print('prec_hyp',prec_hyp)
    f1score_hyp = 2*recl_hyp/(recl_hyp + prec_hyp) if overlap_hyp_cnt > 0 else 0
    print('f1score_hyp',f1score_hyp)
    f1res = (2*f1score+f1score_hyp)/3
    print('f1res',f1res,'\n')
    return (1.0 - f1res)
    

buff = list(res.items())
diverg = np.zeros((len(buff),len(buff)))
for lli,ll in enumerate(buff):
    for rri,rr in enumerate(buff):
        diverg[lli,rri] = divergence(ll[0], rr[0])

Streaming output truncated to the last 5000 lines.
prec_hyp 0.0
f1score_hyp 0
f1res 0.0 

buff_a_first {'conceptualization', 'natural_science', 'situation', 'hypothesis'}
buff_b_first {'trap', 'feature'}
buff_a_second {'science', 'concept', 'creating_by_mental_acts', 'state'}
buff_b_second {'device', 'property'}
recl_hyp 0.0
prec_hyp 0.0
f1score_hyp 0
f1res 0.0 

buff_a_first {'conceptualization', 'natural_science', 'situation', 'hypothesis'}
buff_b_first {'collection', 'pure_mathematics', 'artifact', 'time', 'delay', 'ability'}
buff_a_second {'science', 'concept', 'creating_by_mental_acts', 'state'}
buff_b_second {'whole', 'mathematics', 'group', 'attribute', 'cognition', 'pause'}
recl_hyp 0.0
prec_hyp 0.0
f1score_hyp 0
f1res 0.0 

buff_a_first {'conceptualization', 'natural_science', 'situation', 'hypothesis'}
buff_b_first {'premise', 'native', 'planning', 'man', 'communication', 'gregorian_calendar_month'}
buff_a_second {'science', 'concept', 'creating_by_mental_acts', 'state'}
buff

In [17]:
diverg

array([[0.        , 1.        , 1.        , ..., 0.61111111, 1.        ,
        1.        ],
       [1.        , 0.        , 1.        , ..., 1.        , 1.        ,
        1.        ],
       [1.        , 1.        , 0.        , ..., 1.        , 0.57575758,
        1.        ],
       ...,
       [0.72222222, 1.        , 1.        , ..., 0.        , 1.        ,
        0.91111111],
       [1.        , 1.        , 0.75757576, ..., 1.        , 0.        ,
        1.        ],
       [1.        , 1.        , 1.        , ..., 0.91111111, 1.        ,
        0.        ]])